# Оценка линейного искажающего оператора в задаче восстановления изображений

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import convolve2d as conv2

from skimage import color, data, restoration, io, img_as_float
from skimage.restoration import uft
from scipy.signal import fftconvolve, convolve, convolve2d
from scipy.stats.stats import pearsonr
from numpy.fft import fftn, ifftn
from skimage.measure import compare_psnr
from skimage.draw import bezier_curve

In [ ]:
#interactive graphics
#%matplotlib notebook

In [ ]:
# Prepare data
sz = 11
astro = color.rgb2gray(data.astronaut())

psf = np.ones((sz, sz)) / (sz*sz)
astro_noisy = conv2(astro, psf, 'same')
# Add Noise to Image
#astro_noisy += (np.random.poisson(lam=25, size=astro.shape) - 10) / 255.

In [ ]:
def show_results(original, noisy, result, titles=['Original Data', 'Blurred data', 'Restoration using\nRichardson-Lucy']):
    # Show results
    fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(12, 10))
    plt.gray()

    for a in (ax[0], ax[1], ax[2]):
           a.axis('off')

    ax[0].imshow(original, vmin=0, vmax=1)
    ax[0].set_title(titles[0])

    ax[1].imshow(noisy, vmin=0, vmax=1)
    ax[1].set_title(titles[1])

    ax[2].imshow(result, vmin=0, vmax=1)
    ax[2].set_title(titles[2])


    fig.subplots_adjust(wspace=0.02, hspace=0.2,
                        top=0.9, bottom=0.05, left=0, right=1)
    plt.show()

In [ ]:
def row_correlation(image):
    return pearsonr(image[1:,:].ravel(), image[:-1,:].ravel())[0]
    
def col_correlation(image):
    return pearsonr(image[:,1:].ravel(), image[:,:-1].ravel())[0]

In [ ]:
def desubsample(image, times=1, axises=None):
    if axises is None:
        axises = [i for i, sz in enumerate(image.shape) if sz > 4]
    for axis in axises:
        image = image.repeat(times, axis)#, inplace=True)
    return image

def corelucy(Y, psf, dampar22, wI, readout, subsample, numNSdim, vec, num, eps=1e-9, useFFT=False):
    """
    CORELUCY Accelerated Damped Lucy-Richarson Operator.
    Calculates function that when used with the scaled projected array
    produces the next iteration array that maximizes the likelihood that
    the entire suite satisfies the Poisson statistics.
    """
    if useFFT:
        reBlurred = np.real(ifftn(fftn(Y) * psf)) #Here psf=H = fftn(psf)
    else:
        reBlurred = np.real(convolve2d(Y, psf, 'same'))

    # 1. Resampling if needed
    if subsample != 1: # Bin reBlurred back to the sizeI for non-singleton dims
        #1.Reshape so that the-to-binned dimension separates into two
        #dimensions, with one of them consisting of elements of a single bin.
        reBlurred = np.reshape(reBlurred, vec);

        #2. Bin (==calculate mean) along the first of the-to-binned dimension,
        #that dimension consists of the bin elements. Reshape to get rid off
        for k in num: # new appeared singleton.
            vec.pop(k) #~ vec(k) = [];
            reBlurred = reshape(mean(reBlurred,k),vec);

    # 2. An Estimate for the next step
    reBlurred += readout;
    reBlurred[reBlurred <= 0] = eps;
    AnEstim = wI / reBlurred + eps;
    AnEstim[AnEstim<=0] = eps

    # 3. Damping if needed
    if dampar22 == 0: # No Damping
        ImRatio = desubsample(AnEstim, subsample, numNSdim);
    else: # Damping of the image relative to dampar22 = (N*sigma)^2
        gm = 10;
        g = (wI * np.log(AnEstim)+ reBlurred - wI) / dampar22;
        g = np.minimum(g,1);
        G = (g**(gm-1))*(gm-(gm-1)*g);
        ImRatio = 1 + desubsample(G,subsample, numNSdim) * (desubsample(AnEstim, subsample, numNSdim) - 1);
    if useFFT:
        return fftn(ImRatio);
    else:
        return ImRatio

def richardson_lucy_matlab(image, psf, iterations=50, dampar=0, weight=None, readout=0, 
                           subsample=1, eps=1e-16, clip=True, useFFT=False):
    """ Richardson-Lucy deconvolution.

    Parameters
    ----------
    image : ndarray
       Input degraded image (can be N dimensional).
    psf : ndarray
       The point spread function.
    iterations : int
       Number of iterations. This parameter plays the role of
       regularisation.
    """
    #if 'eps' not in globals():
    #    eps = 1e-9
    # to continue restoration
    sizeI = image.shape
    sizePSF = psf.shape
    numNSdim = [i for i, sz in enumerate(sizePSF) if sz > 4] # do not desubsample rgb axis
    
    psf_mirror = psf[::-1, ::-1]

    assert(not np.all(psf == 0))

    if isinstance(image, list) and len(image) == 4:
        image, prev_image, prev_prev_image, internal = image
    else:
        prev_image = np.ones(image.shape)*.5#image
        prev_prev_image = 0
        internal = np.zeros((image.size*subsample**len(numNSdim),2))
    internal[:,1] = 0

    if weight is None:
        weight = np.ones(image.shape)

    # 1. Prepare OTF
    #~ H = psf2otf(psf, sizeI);
    if psf.shape != image.shape:
        H = uft.ir2tf(psf, image.shape, is_real=False)
    else:
        H = psf
    H_mirror = H[::-1, ::-1]

    wI = np.maximum(weight * (readout + image), 0)
    prev_image = desubsample(prev_image, subsample, numNSdim)
    if useFFT:
        scale = np.real(ifftn(np.conj(H)*fftn(desubsample(weight)))) + np.sqrt(eps)
    else:
        scale = np.real(convolve2d(desubsample(weight), psf_mirror, 'same')) + np.sqrt(eps)
    del weight

    dampar22 = np.square(dampar)/2

    '''
    prepare vector of dimensions to facilitate the reshaping
    % when the matrix is binned within the iterations.
    '''
    vec = []
    num = []
    if subsample != 1:
        for dim, sz in enumerate(sizeI):
            if sz != 1:
                vec.extend((subsample, sz))
                num.append(dim)
            else:
                vec.append(sz)
    num.reverse()

    # 3 L_R Iterations
    lambd = 2 * np.any(internal != 0)
    correlation_X = [col_correlation(image)]
    correlation_Y = [row_correlation(image)]
    for k in range(lambd, lambd + iterations):

        # 3.a Make an image predictions for the next iteration
        if k > 2:
            lambd = (internal[:,0].T.dot(internal[:,1])) / (internal[:,1].T.dot(internal[:,1]) +eps) # (scalar division)
            lambd = max(min(lambd, 1), 0) # stability enforcement saturation
        Y = np.maximum(prev_image + lambd*(prev_image - prev_prev_image), 0) # plus positivity constraint

        # 3.b Make core for the LR estimation
        if useFFT:
            cc = corelucy(Y, H, dampar22, wI, readout, subsample, numNSdim, vec, num, eps, useFFT)
        else:
            cc = corelucy(Y, psf, dampar22, wI, readout, subsample, numNSdim, vec, num, eps, useFFT)

        # 3.c Determine next iteration image and apply poitivity constraint
        prev_prev_image = prev_image
        if useFFT:
            prev_image = np.maximum(Y * np.real(ifftn(cc * np.conj(H))) / scale, 0)
        else:
            prev_image = np.maximum(Y * np.real(convolve2d(cc, psf_mirror, 'same')) / scale, 0)
        if clip:
            prev_image[prev_image > 1] = 1
            prev_image[prev_image < eps] = eps
        correlation_X.append(col_correlation(prev_image))
        correlation_Y.append(row_correlation(prev_image))
        del cc
        internal[:,1] = internal[:,0]
        internal[:,0] = (prev_image-Y).ravel()
    del wI, H, scale, Y
    return {'image': prev_image, 
            'correlationX': np.array(correlation_X)/(image.size-1), 
            'correlationY': np.array(correlation_Y)/(image.size-1)}

In [ ]:
from itertools import chain
def plot_corr(n, ydata, legend=['row correlation', 'column correlation'], title='Normalized correlation'):
    plt.plot(*list(chain(*[(range(n), y) for y in ydata])))
    plt.grid()
    plt.title(title)
    plt.xlabel('n iterations')
    plt.legend(legend)
    plt.show()

In [ ]:
# Restore Image using Richardson-Lucy algorithm:
iterations = 20
deconv = richardson_lucy_matlab(astro_noisy, psf, iterations=iterations, eps=1e-5)
show_results(astro, astro_noisy, deconv['image'])
plot_corr(iterations+1, [deconv['correlationX'], 
                         deconv['correlationY']])

In [ ]:
liftingbody = img_as_float(io.imread('liftingbody.png'))
lifting_blurred = conv2(liftingbody, psf, 'same')

In [ ]:
# Restore Image using Richardson-Lucy algorithm my:
iterations = 20
deconv = richardson_lucy_matlab(lifting_blurred, psf, iterations=iterations, eps=1e-5, useFFT=True)
show_results(liftingbody, lifting_blurred, deconv['image'])
plot_corr(iterations+1, [deconv['correlationX'], 
                         deconv['correlationY']])

// matlab original for gen psf (curved line shift)
```python
def gen_psf():
    p0 = np.random.randint(6, 25)
    p1 = np.random.randint(6, 21)

    x = np.random.randint(0, p0, (1,4))#sort(randi([0 p0], 1, 4));
    y = np.random.randint(0, p1, (1,4))

    pt1 = [x[0]; y[0]]
    pt2 = [x[1]; y[1]]
    pt3 = [x[2]; y[2]]
    pt4 = [x[3]; y[3]]

    t = list(range(0, p0+1))
    pts = kron((1-t).^3,pt1) + kron(3*(1-t).^2.*t,pt2) + kron(3*(1-t).*t.^2,pt3) + kron(t.^3,pt4);
    x1 = 1:p0;
    y1 = round(pts(2, :));
    y1 = y1 - min(y1) + 1;
    %plot(x1,y1);
    pp = max(y1);
    PSF = zeros(p0, pp);

    for ind_x = 1:length(x1)
        yy = max(length(x1) - y1(ind_x), 1);
        PSF(yy, ind_x) = rand();
    end

    PSF = PSF./(sum(PSF(:)) + eps*p0*p0);
```

## Криволинейный оператор смаза

In [ ]:
def curved_psf(points = 3, maxlen = 30):
    eps = 1e-16
    p0 = np.random.randint(6, maxlen)
    p1 = np.random.randint(6, maxlen)

    x = np.random.randint(0, p0, points)#sort(randi([0 p0], 1, 4));
    y = np.random.randint(0, p1, points)

    pt1 = [[0], [0]]#[[x[0]], [y[0]]]
    pt2 = [[x[1]], [y[1]]]
    pt3 = [[x[2]], [y[2]]]
    print('curve', pt1, pt2, pt3)
    #pt4 = [[x[3]], [y[3]]]

    t = np.linspace(0, 1, p0)
    #pts = np.kron((1-t) ** 3, pt1) + np.kron(3*(1-t)**2 * t, pt2) + np.kron(3 * (1-t) * t**2, pt3) + np.kron(t**3, pt4)
    pts = np.kron((1-t)**2, pt1) + np.kron(2*t*(1-t), pt2) + np.kron(t**2, pt3)
    x1 = np.arange(1, p0+1)
    y1 = np.round(pts[1, :]).astype(np.int64)
    y1 = y1 - min(y1) + 1
    #plot(x1,y1)
    pp = np.max(y1)
    PSF = np.zeros((p0, len(x1)))

    print(len(x1), PSF.shape)

    for ind_x in range(len(x1)):
        yy = max(len(x1) - y1[ind_x], 1)
        PSF[yy, ind_x] = np.random.rand()

    PSF = PSF/(sum(PSF.ravel()) + eps*p0*p0)
    return PSF

In [ ]:
psf = curved_psf()
lifting_blurred = conv2(liftingbody, psf, 'same')

In [ ]:
plt.imshow(psf)

In [ ]:
r = restoration.richardson_lucy(lifting_blurred, psf)
plt.imshow(r)

In [ ]:
# Restore Image using Richardson-Lucy algorithm my:
iterations = 3
deconv = richardson_lucy_matlab(lifting_blurred, psf, iterations=iterations, eps=1e-5)
deconv_py = restoration.richardson_lucy(lifting_blurred, psf, iterations=iterations)
show_results(lifting_blurred, deconv_py, deconv['image'], titles=['Blurred data', 'Restoration using\nRichardson-Lucy python', 'Restoration using\nRichardson-Lucy my'])
plot_corr(iterations+1, [deconv['correlationX'], 
                         deconv['correlationY']])

## Линейный оператор смаза

In [ ]:
from skimage.draw import line_aa
from math import sin, cos, pi
def motion_blur_psf(length, angle):
    cc, rr, val = line_aa(0, 0, int(length*cos(angle)), int(length*sin(angle)))
    psf = np.zeros((max(rr)+1, max(cc)+1))
    psf[rr, cc] = val
    #psf[0, 0] = psf[0, 0]/2
    #psf[-1, -1] = psf[-1, -1]/2
    return psf/np.sum(psf)

In [ ]:
from math import sin, cos, pi
"""Incorrectly works with angles k*pi/2, where k is integer."""
def motion_blur_psf_my(length=10, angle=pi/4, n_points=1000, **kwargs):
    x_start, y_start = 0, 0
    if 'x' in kwargs and 'y' in kwargs:
        x_end, y_end = kwargs['x'], kwargs['y']
    else:
        x_end, y_end = length*cos(angle), length*sin(angle)
    if x_end < 0:
        x_start -= x_end
        x_end = 0
    if y_end < 0:
        y_start -= y_end
        y_end = 0
    psf = np.zeros((int(max(y_start, y_end))+2, int(max(x_start, x_end))+2))
    
    triangle_fun = lambda x: np.maximum(0, (1 - np.abs(x)))
    triangle_fun_prod = lambda x, y: np.multiply(triangle_fun(x), triangle_fun(y))
    
    X = np.linspace(x_start, x_end, n_points)
    Y = np.linspace(y_start, y_end, n_points)
    x1 = np.floor(X).astype(np.int)
    x2 = x1+1
    y1 = np.floor(Y).astype(np.int)
    y2 = y1+1
    print(x_start, y_start, x_end,y_end)
    psf[y1, x1] += triangle_fun_prod(X - x1, Y - y1)
    psf[y2, x1] += triangle_fun_prod(X - x1, Y - y2)
    psf[y1, x2] += triangle_fun_prod(X - x2, Y - y1)
    psf[y2, x2] += triangle_fun_prod(X - x2, Y - y2)
    return psf/np.sum(psf)

In [ ]:
np.round(motion_blur_psf_my(x=3,y=-4.5),3)

In [ ]:
shift = 30
psf = motion_blur_psf_my(shift, pi/3)
lifting_blurred = conv2(liftingbody, psf, 'same')
iterations = 40
deconv = richardson_lucy_matlab(lifting_blurred, psf, iterations=iterations, eps=1e-5, clip=False, dampar=0.004)
show_results(liftingbody, lifting_blurred, deconv['image'])
plot_corr(iterations+1, [deconv['correlationX'], 
                         deconv['correlationY']])
correlation_X = pearsonr(liftingbody.ravel('C')[:-1], liftingbody.ravel('C')[1:])
correlation_Y = pearsonr(liftingbody.ravel('F')[:-1], liftingbody.ravel('F')[1:])
print(correlation_X, correlation_Y)

## Неверная psf

In [ ]:
psf_wrong = motion_blur_psf(shift, pi/4)
#iterations = 20
deconv_wrong = richardson_lucy_matlab(lifting_blurred, psf_wrong, iterations=iterations, eps=1e-5)
show_results(liftingbody, lifting_blurred, deconv_wrong['image'])
plot_corr(iterations+1, [deconv['correlationX'], 
                         deconv['correlationY'],
                         deconv_wrong['correlationX'],
                         deconv_wrong['correlationY']], 
          legend=['row correlation', 'column correlation', 'row correlateion(wrong psf)', 'column correlation(wrong psf)'])

## Использование параметра dampar

In [ ]:
def find_noise(image):
    N, M = image.shape
    F = np.fft.fftn(image)
    part = .02
    area = F[int(N/2-N*part):int(N/2+N*part), int(M/2-M*part):int(M/2+M*part)]
    np.std
    np.var
    msv = np.sqrt(np.mean(np.abs(area) ** 2) / (N*M))
    I_res = np.fft.ifftn(F-msv)
    NSR = msv**2 / np.var(I_res)
    return (msv, NSR)
s_n, S_find = find_noise(liftingbody)

In [ ]:
s_n

In [ ]:
psf = motion_blur_psf(shift, pi/4)
lifting_blurred = conv2(liftingbody, psf, 'same')
iterations = 30
deconv = richardson_lucy_matlab(lifting_blurred, psf, iterations=iterations, eps=1e-5, clip=False, dampar=s_n)
show_results(liftingbody, lifting_blurred, deconv['image'])
plot_corr(iterations+1, [deconv['correlationX'], 
                         deconv['correlationY']])

### dampar и неверная psf

In [ ]:
psf_wrong = motion_blur_psf_my(shift, pi/3)
#iterations = 40
deconv_wrong = richardson_lucy_matlab(lifting_blurred, psf_wrong, iterations=iterations, eps=1e-5, dampar=s_n)
show_results(liftingbody, lifting_blurred, deconv_wrong['image'])
plot_corr(iterations+1, [deconv['correlationX'], 
                         deconv['correlationY'],
                         deconv_wrong['correlationX'],
                         deconv_wrong['correlationY']], 
          legend=['row correlation', 'column correlation', 'row correlateion(wrong psf)', 'column correlation(wrong psf)'])

```
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter


w, h = lifting_blurred.shape
x = np.arange(w)
y = np.arange(h)
X, Y = np.meshgrid(x, y)

fig = plt.figure(figsize=(10,10))
ax = fig.gca(projection='3d')
# Plot the surface.
surf = ax.plot_surface(X, Y, F, cmap=cm.coolwarm,
                       linewidth=0, antialiased=False)

# Customize the z axis.
#ax.set_zlim(0, 1.01)
#ax.zaxis.set_major_locator(LinearLocator(10))
#ax.zaxis.set_major_formatter(FormatStrFormatter('%.02f'))

# Add a color bar which maps values to colors.
#fig.colorbar(surf, shrink=0.5, aspect=5)

plt.show()
```

## Кепстр
$$K = F^{-1}\{log(1+\left|F\{I\}\right|)\}$$

In [ ]:
from skimage.filters import gaussian

b_clip = 3
N,M = lifting_blurred.shape
# ( ifft2 (100* log (1+ abs ( fft2 ( I ) ) ) ) ) ;
K = np.fft.ifftn(100*np.log(1+np.abs(np.fft.fftn(lifting_blurred))))#[b_clip:N//2,b_clip:M//2]
K_shift = np.fft.fftshift(K)
#K = gaussian(np.abs(K), 1)

#mask = np.ones((N, M))
#mask[1:3, 1:3] = 0
#K *= mask

In [ ]:
#%matplotlib notebook
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter


h, w = K.shape
x = np.arange(w)
y = np.arange(h)
X, Y = np.meshgrid(x, y)
print(X.shape, Y.shape)
fig = plt.figure(figsize=(10,10))
ax = fig.gca(projection='3d')
# Plot the surface.
H = 1
surf = ax.plot_surface(X, Y, np.abs(K_shift), cmap=cm.coolwarm,
                       linewidth=0, antialiased=False)
plt.xlabel('x')
plt.ylabel('y')
# Customize the z axis.
#ax.set_zlim(0, H + H/20)
#ax.zaxis.set_major_locator(LinearLocator(10))
#ax.zaxis.set_major_formatter(FormatStrFormatter('%.02f'))

# Add a color bar which maps values to colors.
#fig.colorbar(surf, shrink=0.5, aspect=5)

plt.show()

In [ ]:
nn = np.argmin(np.real(K_shift))
n, m, = K_shift.shape
x0 = [nn // n - n//2, nn % n - m//2]

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(np.clip(np.real(K_shift),-1,1))#, vmin=noisy.min(), vmax=noisy.max())
plt.plot(n//2 + x0[0], m//2 + x0[1], 'ro')
plt.title('Кепстр изображения с выделенным минмиумом')
plt.savefig('kepstr.png')
plt.show()

## Уточнение искажающего оператора
$$\varepsilon = ||\tilde{I}\oplus\tilde{h}-I_0|| \to \min_{(x,y)}$$
Пусть $$\tilde{I}\oplus(\tilde{h}+\tilde{dh})=I_0$$
Тогда $$\tilde{I}\oplus\tilde{dh}=I_0-\tilde{I}\oplus\tilde{h}$$
Получим задачу аналогичную исходной($\tilde{I}\oplus h +\eta = I_0$)

```python
img_diff = liftingbody - convolve2d(deconv['image'], psf, mode='same') # Утечка
deconv_psf = richardson_lucy_matlab(img_diff, deconv['image'], iterations=iterations, eps=1e-5, dampar=s_n, useFFT=True)
psf_new = deconv_psf['image']
deconv_upd = richardson_lucy_matlab(lifting_blurred, psf_new, iterations=iterations, eps=1e-5, dampar=s_n, useFFT=True)
show_results(lifting_blurred, deconv['image'], deconv_upd['image'],
             titles=['blurred', 'restored', 'restored with\nnew psf'])
plot_corr(iterations+1, [deconv['correlationX'], 
                         deconv['correlationY'],
                         deconv_upd['correlationX'],
                         deconv_upd['correlationY']], 
          legend=['row correlation', 'column correlation', 'row correlateion(new psf)', 'column correlation(new psf)'])
```

Такой подход не сработал :( Будем использовать метод Ньютона
## Уточнение искажающего оператора

In [ ]:
from functools import partial
def funcToMinimize(xy, I_blurred, *args, **kwargs):
    psf = motion_blur_psf_my(x=xy[0], y=xy[1])
    restored = richardson_lucy_matlab(I_blurred, psf, *args, **kwargs)
    I_restored = restored['image']
    df = convolve2d(I_restored, psf, 'same') - I_blurred
    return np.mean(np.square(df))

In [ ]:
partial(funcToMinimize, I_blurred=lifting_blurred, iterations=iterations, eps=1e-5, dampar=s_n, useFFT=True)([10,10])

In [ ]:
cos(pi/4)*30

### Минимизация методом Нелдера-Мида (симплекс-метод)

In [ ]:
from scipy.optimize import minimize

# 0 0 19.92055100537836 23.428020572755493
# Optimization terminated successfully.
#          Current function value: 0.000656
#          Iterations: 27
#          Function evaluations: 66

# res = minimize(partial(funcToMinimize, I_blurred=lifting_blurred, iterations=iterations, eps=1e-5, dampar=s_n, useFFT=True),
#                x0=x0, method='Nelder-Mead',
#                options={'xtol': 1e-3, 'disp': True})

In [ ]:
%time

In [ ]:
# Метод Пауэлла (метод спуска по векторам с использованием квадратичной аппроксимации)
# Время работы большое(не дождался)

# res_powell = minimize(partial(funcToMinimize, I_blurred=lifting_blurred, iterations=iterations, eps=1e-5, dampar=s_n, useFFT=True),
#                x0=x0, method='Powell',
#                options={'xtol': 1e-3, 'disp': True})
# print(res_powell)

[Метод сопряжённых градиентов](https://ru.wikipedia.org/wiki/%D0%9C%D0%B5%D1%82%D0%BE%D0%B4_%D1%81%D0%BE%D0%BF%D1%80%D1%8F%D0%B6%D1%91%D0%BD%D0%BD%D1%8B%D1%85_%D0%B3%D1%80%D0%B0%D0%B4%D0%B8%D0%B5%D0%BD%D1%82%D0%BE%D0%B2)

In [ ]:
# Метод сопряжённых градиентов
# Очень медленно движется в верном направлении
# 0 0 21.94429490476539 21.999610267537243
# 0 0 21.94429490476539 21.999610267537243
# 0 0 21.94429491966655 21.999610267537243

# res_cg = minimize(partial(funcToMinimize, I_blurred=lifting_blurred, iterations=iterations, eps=1e-5, dampar=s_n, useFFT=True),
#                x0=x0, method='CG')
# print(res_cg)

[BFGS](https://ru.wikipedia.org/wiki/%D0%90%D0%BB%D0%B3%D0%BE%D1%80%D0%B8%D1%82%D0%BC_%D0%91%D1%80%D0%BE%D0%B9%D0%B4%D0%B5%D0%BD%D0%B0_%E2%80%94_%D0%A4%D0%BB%D0%B5%D1%82%D1%87%D0%B5%D1%80%D0%B0_%E2%80%94_%D0%93%D0%BE%D0%BB%D1%8C%D0%B4%D1%84%D0%B0%D1%80%D0%B1%D0%B0_%E2%80%94_%D0%A8%D0%B0%D0%BD%D0%BD%D0%BE)
квазиньютоновский метод

In [ ]:
# Алгоритм Бройдена — Флетчера — Гольдфарба — Шанно (BFGS)
# fun: 0.000661486767220792
#  hess_inv: array([[3.50190492e-09, 4.84852786e-09],
#        [4.84852786e-09, 6.73400920e-09]])
#       jac: array([ 1.071819  , -0.53368984])
#   message: 'Desired error not necessarily achieved due to precision loss.'
#      nfev: 384
#       nit: 4
#      njev: 93
#    status: 2
#   success: False
#         x: array([21.9432086 , 21.99968588])

# res_bfgs = minimize(partial(funcToMinimize, I_blurred=lifting_blurred, iterations=iterations, eps=1e-5, dampar=s_n, useFFT=True),
#                x0=x0, method='BFGS')
# print(res_bfgs)

In [ ]:
# !!! Jacobian is requiered
# res_newton_cg = minimize(partial(funcToMinimize, I_blurred=lifting_blurred, iterations=iterations, eps=1e-5, dampar=s_n, useFFT=True),
#                x0=x0, method='Newton-CG')

L-BFGS-B

In [ ]:
# быстро
# 0 0 22.00000309066789 22.000001525058693
# 0 0 22.00000309066789 22.000001515058692
# 0 0 22.00000310066789 22.000001515058692
# 0 0 22.00000309066789 22.000001525058693
#       fun: 0.0009062932190944391
#  hess_inv: <2x2 LbfgsInvHessProduct with dtype=float64>
#       jac: array([ 1.54585227, 31.06279667])
#   message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
#      nfev: 90
#       nit: 2
#    status: 0
#   success: True
#         x: array([22.00000309, 22.00000152])

# res_l_bfgs_b = minimize(partial(funcToMinimize, I_blurred=lifting_blurred, iterations=iterations, eps=1e-5, dampar=s_n, useFFT=True),
#                x0=x0, method='L-BFGS-B')
# print(res_l_bfgs_b)

TNC - truncated Newton (for optimizing non-linear functions with large numbers of independent variables.)

In [ ]:
from scipy.optimize import minimize
res_tnc = minimize(partial(funcToMinimize, I_blurred=lifting_blurred, iterations=iterations, eps=1e-5, dampar=s_n, useFFT=True),
               x0=x0, method='TNC')
print(res_tnc)

Method Nelder-Mead uses the Simplex algorithm [1], [2]. This algorithm is robust in many applications. However, if numerical computation of derivative can be trusted, other algorithms using the first and/or second derivatives information might be preferred for their better performance in general.

Method Powell is a modification of Powell’s method [3], [4] which is a conjugate direction method. It performs sequential one-dimensional minimizations along each vector of the directions set (direc field in options and info), which is updated at each iteration of the main minimization loop. The function need not be differentiable, and no derivatives are taken.

Method CG uses a nonlinear conjugate gradient algorithm by Polak and Ribiere, a variant of the Fletcher-Reeves method described in [5] pp. 120-122. Only the first derivatives are used.

Method BFGS uses the quasi-Newton method of Broyden, Fletcher, Goldfarb, and Shanno (BFGS) [5] pp. 136. It uses the first derivatives only. BFGS has proven good performance even for non-smooth optimizations. This method also returns an approximation of the Hessian inverse, stored as hess_inv in the OptimizeResult object.

Method Newton-CG uses a Newton-CG algorithm [5] pp. 168 (also known as the truncated Newton method). It uses a CG method to the compute the search direction. See also TNC method for a box-constrained minimization with a similar algorithm. Suitable for large-scale problems.

Method dogleg uses the dog-leg trust-region algorithm [5] for unconstrained minimization. This algorithm requires the gradient and Hessian; furthermore the Hessian is required to be positive definite.

Method trust-ncg uses the Newton conjugate gradient trust-region algorithm [5] for unconstrained minimization. This algorithm requires the gradient and either the Hessian or a function that computes the product of the Hessian with a given vector. Suitable for large-scale problems.

Method trust-krylov uses the Newton GLTR trust-region algorithm [14], [15] for unconstrained minimization. This algorithm requires the gradient and either the Hessian or a function that computes the product of the Hessian with a given vector. Suitable for large-scale problems. On indefinite problems it requires usually less iterations than the trust-ncg method and is recommended for medium and large-scale problems.

Method trust-exact is a trust-region method for unconstrained minimization in which quadratic subproblems are solved almost exactly [13]. This algorithm requires the gradient and the Hessian (which is not required to be positive definite). It is, in many situations, the Newton method to converge in fewer iteraction and the most recommended for small and medium-size problems.

In [ ]:
deconv_orig = richardson_lucy_matlab(lifting_blurred, psf, iterations=iterations, dampar=s_n, useFFT=False)
psnr_orig = compare_psnr(liftingbody, deconv_orig['image'])

psf_init = motion_blur_psf_my(x=x0[0], y=x0[1])
deconv_init = richardson_lucy_matlab(lifting_blurred, psf_init, iterations=iterations, dampar=s_n, useFFT=False)
psnr_init = compare_psnr(liftingbody, deconv_init['image'])

x_found, y_found = res_tnc['x']
psf_found = motion_blur_psf_my(x=x_found, y=y_found)
deconv_found = richardson_lucy_matlab(lifting_blurred, psf_found, iterations=iterations, dampar=s_n, useFFT=False)
psnr_found = compare_psnr(liftingbody, deconv_found['image'])

show_results(deconv_orig['image'], deconv_init['image'], deconv_found['image'],
             titles=['Restored with true psf\nPSNR={0}'.format(psnr_orig), 
                     'With initial approxiamtion\nPSNR={0}'.format(psnr_init),
                     'Minimized error\nPSNR={0}'.format(psnr_found)])

## Оценка криволинейного искажающего оператора

In [ ]:
from skimage.draw import line_aa

In [ ]:
sz = 20
r, c = bezier_curve(0, 0, sz//10, sz-1 - sz//10, sz-1, sz-1, weight=1)
#r, c, val = line_aa(0, 0, 15, 19)
#r, c = bezier_curve(0, 0, sz//2, sz//2, sz-1, sz-1, weight=1)
psf_bezier = np.zeros((sz, sz))
psf_bezier[r, c] = 1
psf_bezier /= psf_bezier.sum()

iterations = 4

In [ ]:
psf_bezier.round(2)

In [ ]:
lifting_bezier_blurred = convolve2d(liftingbody, psf_bezier, 'same')
#lifting_restored = restoration.richardson_lucy(lifting_bezier_blurred, psf_bezier, iterations=iterations)
deconv_bezier = richardson_lucy_matlab(lifting_bezier_blurred, psf_bezier, iterations=iterations, clip=True, useFFT=False)

In [ ]:
print('iterations = {0}'.format(iterations))
#show_results(lifting_bezier_blurred, lifting_restored, deconv_bezier['image'],
#            titles = ['Blurred data', 'Restored with python LR', 'Restored with my LR'])
show_results(liftingbody, lifting_bezier_blurred, deconv_bezier['image'])
plot_corr(iterations+1, [deconv_bezier['correlationX'], 
                         deconv_bezier['correlationY']])

In [ ]:
deconv_bezier['image']

Sample log
```
0 0 21.0 21.0
0 0 22.05 21.0
0 0 21.0 22.05
0 0 19.95 22.049999999999997
0 0 21.525 21.2625
0 0 21.525 20.212500000000002
0 0 21.13125 21.590625000000003
0 0 21.65625 21.853125000000006
0 0 21.4921875 21.639843750000004
0 0 21.885937499999997 21.311718749999997
0 0 21.697265624999996 21.3814453125
0 0 21.664453125 21.758789062500004
0 0 21.55986328125 21.386572265625
0 0 21.5947265625 21.51064453125
0 0 21.50859375 21.451171875
0 0 21.5783203125 21.699316406250006
0 0 21.475781249999997 21.82851562500001
0 0 21.564990234375 21.5901123046875
0 0 21.53525390625 21.669580078125005
0 0 21.54345703125 21.575244140625003
0 0 21.483984375 21.734179687500003
0 0 21.498852539062497 21.694445800781253
0 0 21.4557861328125 21.664709472656252
0 0 21.515386962890624 21.668362426757817
0 0 21.52205200195312 21.722964477539065
0 0 21.49965362548828 21.66062393188477
0 0 21.50711975097656 21.681404113769535
0 0 21.503787231445312 21.65410308837891
0 0 21.51205444335937 21.641061401367192
0 0 21.50835342407226 21.67131843566895
0 0 21.509587097167966 21.661232757568364
0 0 21.511253356933594 21.674883270263678
0 0 21.513720703124996 21.654711914062503
0 0 21.519520568847653 21.661841583251956
0 0 21.524487304687497 21.662145996093756
0 0 21.526153564453125 21.67579650878907
0 0 21.51682891845703 21.659983062744146
0 0 21.51993713378906 21.665254211425786
0 0 21.519104003906246 21.65842895507813
0 0 21.523654174804687 21.6553207397461
0 0 21.52086639404297 21.662770843505864
0 0 21.52272491455078 21.65780410766602
0 0 21.51734161376953 21.65408706665039
0 0 21.522700881958006 21.660131263732914
0 0 21.520914459228514 21.658116531372073
0 0 21.521795654296874 21.660287475585942
0 0 21.518222808837884 21.65625801086426
0 0 21.516412353515612 21.656570434570312
0 0 21.51978893280029 21.657730007171633
0 0 21.518663406372063 21.657343482971193
0 0 21.52000923156738 21.6582727432251
0 0 21.517758178710928 21.65749969482422
0 0 21.518198776245107 21.658585166931154
0 0 21.518547248840324 21.657653903961183
0 0 21.519893074035643 21.65858316421509
0 0 21.518291902542106 21.65777056217194
Optimization terminated successfully.
         Current function value: 0.000638
         Iterations: 23
         Function evaluations: 56
```